# Setup

In [ ]:
%pip install kornia==0.4.0
import torch
from torchvision import datasets, transforms
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import copy
import pycocotools

## Mount drive

In [ ]:
your_path = ''
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/'+your_path
import sys
sys.path.append(path)

## GPU

In [ ]:
import torch
train_on_gpu = torch.cuda.is_available()
print(torch.__version__)
if not train_on_gpu:
  DEVICE = 'cpu'
  print('CUDA is not available.  Training on CPU ...')
else:
  DEVICE = 'cuda'
  print('CUDA is available!  Training on GPU ...')

1.7.1
CUDA is not available.  Training on CPU ...


/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# Data analysis

###synthetic

####def synthetic functions

In [ ]:
from datasets.synthetic_shapes.synthetic_shapes_functions import *

####def transforms

In [ ]:
from torchvision import datasets, transforms
import PIL
import torch
from torch.utils.data import Dataset, DataLoader
import os
from skimage import io, transform
import numpy as np
import pandas as pd
import kornia as K

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        T = transforms.ToTensor()
        image = T(image)
        if not landmarks is 0:
          landmarks = torch.tensor(landmarks,dtype=torch.float32)
        return (image,landmarks)


class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        
        img = skimage.transform.resize(image, (new_h, new_w))
        
        landmarks[:,:2] = landmarks[:,:2] * [new_w / w, new_h / h]
        landmarks = landmarks.astype(int)
        return (img,landmarks)
        

class ToGray(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample
        G = transforms.Grayscale(num_output_channels=1)
        if image.size()[0]>1:
          image = G(image)
        return (image,landmarks)   



class ColorJitter(object):
    """Randomly change the brightness, contrast and saturation of an image."""

    def __call__(self, sample):
          image, landmarks = sample
          col = transforms.ColorJitter(brightness=(0.5,1.4))
          image = col(image)
          return (image,landmarks)   



class GaussianBlur(object):
    """Blurs image with randomly chosen Gaussian blur"""

    def __call__(self, sample):
          image, landmarks = sample
          gaus = transforms.GaussianBlur(9, sigma=(1, 1.8))
          image = gaus(image)
          return (image,landmarks)   



#### def dataset

In [ ]:
import skimage

drawing_primitives = [
          'draw_lines',
          'draw_polygon',
          'draw_multiple_polygons',
          'draw_ellipses',
          'draw_star',
          'draw_checkerboard',
          'draw_stripes',
          'draw_cube']



class mySyntheticShapes(Dataset):
    def __init__(self, config, num_imgs_returned, transform=None):
        self.config = config 
        self.num_imgs_returned = num_imgs_returned
        self.transform = transform

    def __len__(self):
      '''Denotes the total number of samples'''
      return self.num_imgs_returned

    def __getitem__(self, index):
          i=0
          while True:
            try:
              primitives = parse_primitives(self.config['primitives'], drawing_primitives)
              primitive = np.random.choice(primitives)
              #print('primitive = ', primitive)

              image = generate_background(self.config['generation']['image_size'],
                                      **self.config['generation']['params']['generate_background'])
              image = add_gausian_noise(image)

              primitive_func_str = primitive+'(image, **self.config["generation"]["params"].get(primitive, {}))'
              points = np.array(eval(primitive_func_str))
              points = pad_points(points, desired_num_points=60)

              sample = (image,points)
              #print('image shape = ', image.shape, 'points shape =', points.shape, 'pts_num=',pts_num)

              if self.transform:
                    sample = self.transform(sample)

            except:
              print("An exception occurred")
              i+=1
              if i<10:
                continue
            break
          return sample

transform = transforms.Compose([Rescale((240,320)),
                                ToTensor(),
                                ToGray()])





####load dataset

In [ ]:
import csv
config = {
          'primitives': 'all',
          'truncate': {},
          'validation_size': -1,
          'test_size': -1,
          'on-the-fly': False,
          'cache_in_memory': False,
          'suffix': None,
          'add_augmentation_to_test_set': False,
          'num_parallel_calls': 10,
          'generation': {
              'split_sizes': {'training': 10000, 'validation': 200, 'test': 500},
              'image_size': [960,1280], 
              'random_seed': 0,
              'params': {
                  'generate_background': {
                      'min_kernel_size': 150, 'max_kernel_size': 500,
                      'min_rad_ratio': 0.02, 'max_rad_ratio': 0.031}, 
                  'draw_stripes': {'transform_params': (0.1, 0.1)},
                  'draw_multiple_polygons': {'kernel_boundaries': (50, 100)}
              },
          },
          'preprocessing': {
              'resize': [240, 320],
              'blur_size': 11,
          },
          'augmentation': {
              'photometric': {
                  'enable': False,
                  'primitives': 'all',
                  'params': {},
                  'random_order': True,
              },
              'homographic': {
                  'enable': False,
                  'params': {},
                  'valid_border_margin': 0,
              },
          }
  }


num_imgs_returned=100000
dataset = mySyntheticShapes(config, num_imgs_returned, transform)
dataloader = DataLoader(dataset, batch_size=1, 
                        shuffle=True, num_workers=0)


#start generating data
filename = path+'/datasets/synthetic_shapes/syn_shape_labels.csv'
im_path = path+'/datasets/synthetic_shapes/synthetic_dataset/'
with open(filename, 'w') as csvfile:  
    #creating a csv writer object  
    csvwriter = csv.writer(csvfile)
    #write headline
    head = np.array(['x{},y{},conf{}'.format(i,i,i).split(',') for i in range(60)]).reshape((1,-1))
    head = np.concatenate((np.array([['im_name']]),head), axis=1)
    csvwriter.writerows(head)
    
    for iter, (im,label) in enumerate(dataloader):
      print(iter)
      full_cord = torch.flatten(label)
      full_cord = np.asarray(full_cord)
      
      name = 'syn_shape_'+str(iter)+'.jpg'
      full_cord = full_cord.reshape((1,-1))
      new_cords = np.concatenate(([[name]],full_cord),axis=1)
      csvwriter.writerows(new_cords)
      torchvision.utils.save_image(im,im_path+name)
